## Chapter 6: Messaging and Event‑Driven Architecture

Modern distributed systems often rely on **messaging** to decouple services, improve resilience, and enable asynchronous processing. Instead of one service calling another directly via HTTP (synchronous communication), services can publish **events** to a message broker, and other services can consume those events at their own pace. This pattern is known as **event‑driven architecture**.

In this chapter, we’ll explore how .NET Aspire simplifies integrating message brokers like RabbitMQ and Azure Service Bus. You’ll learn how to model these resources in the AppHost, connect them to your services using components, and implement a complete order processing workflow. By the end, you’ll be able to build decoupled, scalable systems with ease.

---

### 6.1 Why Messaging?

In our e‑commerce example so far, the web frontend calls the API directly (HTTP). That’s fine for simple requests, but consider what happens when a customer places an order:

- The API must validate the order.
- It must update inventory.
- It must send a confirmation email.
- It might need to notify a shipping service.

If we do all this synchronously in the HTTP request, the customer waits a long time, and any failure (like the email server being down) could cause the entire order to fail. With messaging, the API can publish an `OrderCreated` event and return immediately. Other services (inventory, email, shipping) consume that event asynchronously. This decouples the services and makes the system more resilient.

Key benefits:

- **Decoupling**: Services don’t need to know about each other.
- **Resilience**: If a consumer is down, messages wait in the queue until it recovers.
- **Scalability**: You can run multiple instances of a consumer to process messages in parallel.
- **Flexibility**: New consumers can be added without changing the publisher.

---

### 6.2 Messaging Patterns

Two common patterns are:

- **Queues**: A point‑to‑point channel. A message is sent to a queue and consumed by exactly one consumer (competing consumers pattern).
- **Topics/Exchanges**: A publish/subscribe channel. A message is published to a topic and delivered to all subscribers.

RabbitMQ uses **exchanges** and **queues**; Azure Service Bus uses **queues** and **topics/subscriptions**. Aspire components support both.

---

### 6.3 Recap: Adding RabbitMQ to the AppHost

In Chapter 4, we added RabbitMQ to our solution. Let’s recap the AppHost configuration with a bit more detail.

First, ensure you have the `Aspire.Hosting.RabbitMQ` package installed in the AppHost project:

```bash
cd MyAspireApp.AppHost
dotnet add package Aspire.Hosting.RabbitMQ
```

Now in `Program.cs`, add a RabbitMQ resource:

```csharp
using Aspire.Hosting;

var builder = DistributedApplication.CreateBuilder(args);

// Add RabbitMQ container
var messaging = builder.AddRabbitMQ("messaging")
    .WithManagementPlugin(); // Enables the RabbitMQ management UI at port 15672

// Existing resources
var postgres = builder.AddPostgres("postgres").WithDataVolume();
var productsDb = postgres.AddDatabase("productsdb");

var apiService = builder.AddProject<Projects.MyAspireApp_ApiService>("apiservice")
    .WithReference(productsDb)
    .WithReference(messaging);   // Give the API a connection to RabbitMQ

var workerService = builder.AddProject<Projects.MyAspireApp_WorkerService>("workerservice")
    .WithReference(messaging);   // Worker also needs RabbitMQ

var webfrontend = builder.AddProject<Projects.MyAspireApp_Web>("webfrontend")
    .WithReference(apiService);

builder.Build().Run();
```

**Explanation**:
- `AddRabbitMQ("messaging")` adds a container running the RabbitMQ image. The logical name is `"messaging"`.
- `WithManagementPlugin()` adds the management UI (useful for development). The UI is available at `http://localhost:15672` (default credentials: guest/guest).
- `WithReference(messaging)` injects the RabbitMQ connection string into both the API and worker services as `ConnectionStrings__messaging`.

---

### 6.4 Adding the RabbitMQ Client Component

To actually use RabbitMQ in your services, you need the client component. In both the API and worker projects, add the `Aspire.RabbitMQ.Client` package:

```bash
cd ../MyAspireApp.ApiService
dotnet add package Aspire.RabbitMQ.Client

cd ../MyAspireApp.WorkerService
dotnet add package Aspire.RabbitMQ.Client
```

In each service’s `Program.cs`, add the RabbitMQ client registration:

```csharp
// In ApiService/Program.cs and WorkerService/Program.cs
builder.AddRabbitMQClient("messaging");
```

This method:

- Reads the connection string from configuration (injected by the AppHost).
- Registers `IConnection` and `IChannel` (or `IAsyncConnectionFactory` etc.) in DI.
- Adds health checks (a check that can connect to RabbitMQ).
- Adds OpenTelemetry instrumentation (traces for publish/consume operations, metrics for queue sizes etc.).
- Optionally configures resilience.

Now your services can inject `IConnection` and start publishing or consuming messages.

---

### 6.5 Implementing an Order Processing Workflow

Let’s build a realistic order processing flow:

1. The API receives an order (via a new `POST /orders` endpoint).
2. It saves the order to the database.
3. It publishes an `OrderCreated` message to RabbitMQ.
4. The worker service consumes `OrderCreated` messages and processes them (e.g., updates inventory, sends email).

#### 6.5.1 Define the Order Model and Database

In the API service, add an `Order` class and an `OrderItem` class:

```csharp
// ApiService/Models/Order.cs
namespace MyAspireApp.ApiService.Models;

public class Order
{
    public int Id { get; set; }
    public string CustomerName { get; set; } = string.Empty;
    public DateTime OrderDate { get; set; }
    public decimal TotalAmount { get; set; }
    public List<OrderItem> Items { get; set; } = new();
}

public class OrderItem
{
    public int Id { get; set; }
    public int ProductId { get; set; }
    public int Quantity { get; set; }
    public decimal UnitPrice { get; set; }
}
```

We’ll need a table for orders. Extend the database initialization in `Program.cs`:

```csharp
using var scope = app.Services.CreateScope();
var dataSource = scope.ServiceProvider.GetRequiredService<NpgsqlDataSource>();
using var connection = await dataSource.OpenConnectionAsync();

// Create Products table (if not exists) - already there
await connection.ExecuteAsync(@"
    CREATE TABLE IF NOT EXISTS Products (
        Id SERIAL PRIMARY KEY,
        Name TEXT NOT NULL,
        Price DECIMAL NOT NULL
    )");

// Create Orders and OrderItems tables
await connection.ExecuteAsync(@"
    CREATE TABLE IF NOT EXISTS Orders (
        Id SERIAL PRIMARY KEY,
        CustomerName TEXT NOT NULL,
        OrderDate TIMESTAMP NOT NULL,
        TotalAmount DECIMAL NOT NULL
    )");

await connection.ExecuteAsync(@"
    CREATE TABLE IF NOT EXISTS OrderItems (
        Id SERIAL PRIMARY KEY,
        OrderId INTEGER REFERENCES Orders(Id) ON DELETE CASCADE,
        ProductId INTEGER NOT NULL,
        Quantity INTEGER NOT NULL,
        UnitPrice DECIMAL NOT NULL
    )");
```

#### 6.5.2 Create the Order Endpoint in the API

Add a new endpoint to create an order. We’ll also need a DTO for the request:

```csharp
// ApiService/OrderDto.cs
public record CreateOrderRequest(
    string CustomerName,
    List<OrderItemDto> Items
);

public record OrderItemDto(
    int ProductId,
    int Quantity
);
```

Now in `Program.cs`, add the endpoint:

```csharp
app.MapPost("/orders", async (CreateOrderRequest request, NpgsqlDataSource dataSource, IConnection connection) =>
{
    // 1. Save order to database
    decimal totalAmount = 0;
    using var dbConnection = await dataSource.OpenConnectionAsync();
    using var transaction = await dbConnection.BeginTransactionAsync();

    try
    {
        // Insert order
        var orderId = await dbConnection.ExecuteScalarAsync<int>(@"
            INSERT INTO Orders (CustomerName, OrderDate, TotalAmount)
            VALUES (@CustomerName, @OrderDate, @TotalAmount)
            RETURNING Id",
            new { request.CustomerName, OrderDate = DateTime.UtcNow, TotalAmount = 0m });

        // Insert order items and calculate total
        foreach (var item in request.Items)
        {
            // Get current product price from Products table (simplified)
            var price = await dbConnection.ExecuteScalarAsync<decimal>(
                "SELECT Price FROM Products WHERE Id = @ProductId", new { item.ProductId });
            
            var itemTotal = price * item.Quantity;
            totalAmount += itemTotal;

            await dbConnection.ExecuteAsync(@"
                INSERT INTO OrderItems (OrderId, ProductId, Quantity, UnitPrice)
                VALUES (@OrderId, @ProductId, @Quantity, @UnitPrice)",
                new { OrderId = orderId, item.ProductId, item.Quantity, UnitPrice = price });
        }

        // Update order total
        await dbConnection.ExecuteAsync("UPDATE Orders SET TotalAmount = @TotalAmount WHERE Id = @Id",
            new { TotalAmount = totalAmount, Id = orderId });

        await transaction.CommitAsync();

        // 2. Publish OrderCreated message
        var channel = await connection.CreateChannelAsync();
        await channel.ExchangeDeclareAsync("order-exchange", ExchangeType.Topic, durable: true);
        await channel.QueueDeclareAsync("order-created-queue", durable: true, exclusive: false, autoDelete: false);
        await channel.QueueBindAsync("order-created-queue", "order-exchange", "order.created");

        var message = new OrderCreatedEvent
        {
            OrderId = orderId,
            CustomerName = request.CustomerName,
            TotalAmount = totalAmount,
            Items = request.Items.Select(i => new OrderItemEvent
            {
                ProductId = i.ProductId,
                Quantity = i.Quantity
            }).ToList()
        };

        var body = Encoding.UTF8.GetBytes(JsonSerializer.Serialize(message));
        await channel.BasicPublishAsync(
            exchange: "order-exchange",
            routingKey: "order.created",
            body: body);

        return Results.Ok(new { OrderId = orderId });
    }
    catch (Exception)
    {
        await transaction.RollbackAsync();
        throw;
    }
});
```

We also need the event classes:

```csharp
// ApiService/OrderCreatedEvent.cs
public class OrderCreatedEvent
{
    public int OrderId { get; set; }
    public string CustomerName { get; set; } = string.Empty;
    public decimal TotalAmount { get; set; }
    public List<OrderItemEvent> Items { get; set; } = new();
}

public class OrderItemEvent
{
    public int ProductId { get; set; }
    public int Quantity { get; set; }
}
```

**Explanation**:
- The endpoint first saves the order and its items inside a database transaction. This ensures data integrity.
- After successful commit, it publishes an `OrderCreatedEvent` to RabbitMQ.
- We declare an exchange (`order-exchange`) and a queue (`order-created-queue`) bound with routing key `order.created`. In production, you’d likely manage these separately, but for development this is fine.
- The message is serialized to JSON and published.

#### 6.5.3 Implement the Worker to Consume Messages

In the worker service, we’ll create a background service that consumes messages from the queue. Replace the existing `Worker.cs` with a robust consumer.

First, add a reference to `System.Text.Json` if not already there. Then:

```csharp
// WorkerService/OrderConsumer.cs
using System.Text;
using System.Text.Json;
using RabbitMQ.Client;
using RabbitMQ.Client.Events;

namespace MyAspireApp.WorkerService;

public class OrderConsumer : BackgroundService
{
    private readonly IConnection _connection;
    private readonly ILogger<OrderConsumer> _logger;
    private IChannel? _channel;

    public OrderConsumer(IConnection connection, ILogger<OrderConsumer> logger)
    {
        _connection = connection;
        _logger = logger;
    }

    protected override async Task ExecuteAsync(CancellationToken stoppingToken)
    {
        // Create a channel
        _channel = await _connection.CreateChannelAsync(cancellationToken: stoppingToken);

        // Declare exchange and queue (same as publisher)
        await _channel.ExchangeDeclareAsync("order-exchange", ExchangeType.Topic, durable: true, cancellationToken: stoppingToken);
        await _channel.QueueDeclareAsync("order-created-queue", durable: true, exclusive: false, autoDelete: false, cancellationToken: stoppingToken);
        await _channel.QueueBindAsync("order-created-queue", "order-exchange", "order.created", cancellationToken: stoppingToken);

        // Set up consumer
        var consumer = new AsyncEventingBasicConsumer(_channel);
        consumer.ReceivedAsync += async (model, ea) =>
        {
            try
            {
                var body = ea.Body.ToArray();
                var messageJson = Encoding.UTF8.GetString(body);
                var orderEvent = JsonSerializer.Deserialize<OrderCreatedEvent>(messageJson);

                _logger.LogInformation("Processing order {OrderId} for {Customer}", orderEvent?.OrderId, orderEvent?.CustomerName);

                // Simulate work (e.g., update inventory, send email)
                await Task.Delay(1000, stoppingToken);

                // Acknowledge message
                await _channel.BasicAckAsync(ea.DeliveryTag, multiple: false, cancellationToken: stoppingToken);
                _logger.LogInformation("Order {OrderId} processed successfully", orderEvent?.OrderId);
            }
            catch (Exception ex)
            {
                _logger.LogError(ex, "Error processing message. Will reject and requeue.");
                // Reject and requeue (you might want to dead-letter after retries)
                await _channel.BasicNackAsync(ea.DeliveryTag, multiple: false, requeue: true, cancellationToken: stoppingToken);
            }
        };

        // Start consuming
        await _channel.BasicConsumeAsync("order-created-queue", autoAck: false, consumer: consumer, cancellationToken: stoppingToken);

        // Keep the service running
        await Task.Delay(Timeout.Infinite, stoppingToken);
    }

    public override async Task StopAsync(CancellationToken cancellationToken)
    {
        if (_channel != null && _channel.IsOpen)
        {
            await _channel.CloseAsync(cancellationToken: cancellationToken);
        }
        await base.StopAsync(cancellationToken);
    }
}
```

We also need the `OrderCreatedEvent` class in the worker project (copy it or share via a common library). For simplicity, copy the class definition.

Register this consumer in `Program.cs` of the worker service:

```csharp
using MyAspireApp.ServiceDefaults;
using MyAspireApp.WorkerService;

var builder = Host.CreateApplicationBuilder(args);
builder.AddServiceDefaults();

builder.AddRabbitMQClient("messaging");

// Register the consumer as a hosted service
builder.Services.AddHostedService<OrderConsumer>();

var host = builder.Build();
host.Run();
```

#### 6.5.4 Run and Test

Run the AppHost. In the dashboard, you’ll see the API, worker, PostgreSQL, and RabbitMQ resources. Use the RabbitMQ management UI (`http://localhost:15672`) to inspect queues.

Now let’s create an order. First, ensure there are some products in the database (you can use the `POST /products` endpoint we created earlier). Then send a `POST /orders` request with a JSON body:

```json
{
  "customerName": "John Doe",
  "items": [
    { "productId": 1, "quantity": 2 }
  ]
}
```

You should receive an `OK` response with an order ID. Check the worker’s logs in the Aspire Dashboard—you’ll see the “Processing order” log, and after a second, “Order processed”. In the RabbitMQ management UI, you’ll see the queue depth go to 0 and then back to 0 as the message is consumed.

If you stop the worker service (in the dashboard, you can stop the resource), and then publish an order, the message will remain in the queue. When you restart the worker, it will process the pending message. This demonstrates resilience.

---

### 6.6 Advanced Messaging with Azure Service Bus

While RabbitMQ is great for on‑premises or containerized environments, many cloud‑native applications use Azure Service Bus. Aspire provides components for Service Bus as well.

#### 6.6.1 Adding Azure Service Bus to the AppHost

First, add the hosting package:

```bash
cd MyAspireApp.AppHost
dotnet add package Aspire.Hosting.Azure.ServiceBus
```

Now you can add a Service Bus resource. For local development, you can use the emulator (which runs in a container) or connect to a real Azure Service Bus namespace.

```csharp
var serviceBus = builder.AddAzureServiceBus("servicebus")
    .RunAsEmulator();  // Runs the Service Bus emulator container
```

Or if you want to use a real Azure namespace:

```csharp
var serviceBus = builder.AddAzureServiceBus("servicebus");
```

Then reference it from your services:

```csharp
apiService.WithReference(serviceBus);
workerService.WithReference(serviceBus);
```

The connection string will be injected as `ConnectionStrings__servicebus`.

#### 6.6.2 Using the Service Bus Component

Add the client component to your projects:

```bash
dotnet add package Aspire.Azure.Messaging.ServiceBus
```

In `Program.cs`:

```csharp
builder.AddAzureServiceBusClient("servicebus");
```

This registers `ServiceBusClient` in DI.

Now you can publish messages to a queue or topic. For example, in the API:

```csharp
app.MapPost("/orders", async (CreateOrderRequest request, NpgsqlDataSource dataSource, ServiceBusClient serviceBusClient) =>
{
    // ... save order to database ...

    var sender = serviceBusClient.CreateSender("orders");
    var message = new ServiceBusMessage(JsonSerializer.Serialize(orderEvent))
    {
        ContentType = "application/json",
        MessageId = Guid.NewGuid().ToString()
    };
    await sender.SendMessageAsync(message);

    return Results.Ok(new { OrderId = orderId });
});
```

And in the worker, you’d use `ServiceBusProcessor` to receive messages. The component automatically enables health checks, tracing, and metrics for Service Bus.

---

### 6.7 Messaging Patterns in Depth

#### 6.7.1 Competing Consumers

By default, when you have multiple instances of a consumer service, they all listen to the same queue. RabbitMQ (and Service Bus) will deliver each message to only one consumer. This is the **competing consumers** pattern, which allows you to scale out processing.

If you run multiple instances of the worker (in the AppHost, you can set replicas), you’ll see messages distributed among them.

#### 6.7.2 Publish/Subscribe with Topics

Sometimes you want multiple independent consumers to each receive a copy of a message (e.g., an “order placed” event might go to an analytics service and a notification service). This is publish/subscribe.

In RabbitMQ, you use a **topic exchange** and bind each consumer’s queue with the same routing key. Each consumer gets its own queue, and the exchange copies the message to all bound queues.

In the example above, we used a topic exchange and a single queue. To achieve pub/sub, you’d create a separate queue for each consumer and bind them with the same routing key. The worker would then declare its own queue (perhaps with a unique name) and bind it.

#### 6.7.3 Dead‑Letter Queues

Messages that cannot be processed (e.g., after multiple retries) should be moved to a **dead‑letter queue** for later inspection. RabbitMQ and Service Bus both support this. You can configure the component to handle retries and eventually dead‑letter.

---

### 6.8 Observability and Health Checks

One of the great benefits of Aspire components is that they automatically add telemetry. For RabbitMQ, you’ll see:

- **Traces** for publish and consume operations (with RabbitMQ-specific attributes).
- **Metrics** like the number of messages published, consumed, and queue sizes (if you enable the RabbitMQ management plugin and the component’s metrics).
- **Health checks**: The component adds a health check that verifies connectivity to RabbitMQ. This will appear in the `/health` endpoint and in the dashboard.

Similarly for Azure Service Bus, you get traces and metrics.

---

### 6.9 Hands‑on: Extend with a Second Consumer

Let’s add another worker service that listens for the same `OrderCreated` events but does something different—for example, sends a confirmation email. This will demonstrate the publish/subscribe pattern.

#### Step 1: Create a new worker project

```bash
dotnet new worker -n MyAspireApp.EmailWorker
dotnet sln add MyAspireApp.EmailWorker
cd MyAspireApp.EmailWorker
dotnet add reference ../MyAspireApp.ServiceDefaults
dotnet add package Aspire.RabbitMQ.Client
```

#### Step 2: Add the project to AppHost

In AppHost `Program.cs`, add:

```csharp
var emailWorker = builder.AddProject<Projects.MyAspireApp_EmailWorker>("emailworker")
    .WithReference(messaging);
```

#### Step 3: Implement the email worker

Copy the `OrderConsumer` code but change the logging to simulate sending an email. Also, give it a unique queue name so it gets its own copy of messages. In the `ExecuteAsync` method, declare a queue with a different name, but bind it with the same routing key.

```csharp
await _channel.QueueDeclareAsync("email-notification-queue", durable: true, exclusive: false, autoDelete: false);
await _channel.QueueBindAsync("email-notification-queue", "order-exchange", "order.created");
```

Now when you publish an order, both workers will receive the message (each from its own queue). Check the logs to see both processing the same order ID.

#### Step 4: Observe

In the dashboard, you’ll see both workers consuming. In RabbitMQ management, you’ll see two queues, each with one message (which gets consumed). This is pub/sub in action.

---

### 6.10 Summary

In this chapter, you’ve learned how to build event‑driven systems with .NET Aspire. Key takeaways:

- Messaging decouples services and improves resilience.
- Aspire provides hosting packages to add RabbitMQ and Azure Service Bus to the AppHost.
- The client components automatically register clients, health checks, and telemetry.
- You implemented a complete order processing workflow with a publisher (API) and a consumer (worker).
- You explored the competing consumers and publish/subscribe patterns.
- You saw how to extend with additional consumers and how to use the Azure Service Bus component.

Messaging is a fundamental pattern in cloud‑native applications. With Aspire, integrating a message broker becomes almost trivial, allowing you to focus on business logic.

In the next chapter, we’ll dive into **Externalizing Configuration with Aspire**. You’ll learn how to use Azure App Configuration and Key Vault to manage configuration and secrets across environments, and how to set them up in the AppHost.

---

**Exercises**

1. Modify the worker to simulate a transient failure (throw an exception randomly). Observe how the message is nacked and requeued, and how many retries happen before it possibly gets stuck.
2. Add a dead‑letter queue configuration to RabbitMQ. After three failed attempts, move the message to a DLQ.
3. Replace RabbitMQ with Azure Service Bus (using the emulator) and adapt the code to use `ServiceBusProcessor`. Compare the complexity.
4. Scale the worker to two replicas in the AppHost (use `.WithReplicas(2)`) and observe how messages are load‑balanced.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='5. net_aspire_components.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='7. externalizing_configuration_with_aspire.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
